This notebook contains some functions to run a list of commands and parse the ouput of `time`-ing it.
It requires Jupyter, matplotlib and LaTeX (for exporting the resulting plot).

In [1]:
import subprocess

In [2]:
def ssv_to_dict(ssv):
    """
    Convert a 2-column space-separated table into a dict.
    """
    data = dict()
    for line in ssv.splitlines():
        parts = line.split(" ")
        if len(parts) == 2:
            data[parts[0]] = parts[1]
    return data

def time_command(command):
    """
    Run the given command, time it using `time` and return the results as a dictionary of floats.
    """
    
    proc = subprocess.run([
        "bash", "-c",
        "time -p {}".format(command)
    ], capture_output=True)
    
    raw_dict = ssv_to_dict(proc.stderr.decode("utf-8"))
    
    return {
        key: float(value.replace(",", ".")) for key, value in raw_dict.items()
    }

In [3]:
time_command("mpirun --host localhost:4 ./hello")

{'real': 7.64, 'user': 28.13, 'sys': 0.12}

In [4]:
def time_tasks(tasks):
    results = []
    for label, command in tasks:
        results.append((label, time_command(command)))
    return results

In [5]:
tasks = []
for n_cores in [1,2,4,8,16,32,64]:
    tasks.append((
        "{}".format(n_cores),
        "mpirun --host localhost:{} ./hello".format(n_cores)
    ))

results = time_tasks(tasks)

In [7]:
import matplotlib
import matplotlib.pyplot as plt

# https://timodenk.com/blog/exporting-matplotlib-plots-to-latex/
matplotlib.use("pgf")
matplotlib.rcParams.update({
    "pgf.texsystem": "pdflatex",
    'font.family': "serif",
    'text.usetex': True,
    'pgf.rcfonts': False,
})

labels = [r[0] for r in results]
reals = [r[1]['real'] for r in results]
users = [r[1]['user'] for r in results]

fig, ax = plt.subplots()

ax.plot(labels, reals, users)
ax.set_xlabel('number of cores')
ax.set_ylabel('time elapsed (s)')
ax.set_ylim(bottom=0)
ax.legend(['real', 'user'])

# About right for mtheme beamer slides
fig.set_size_inches(w=4.4, h=2)
fig.savefig('perf.pgf')